In [35]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
load_dotenv()
api_key = os.getenv('API_KEY')
if not api_key:
    print('no api key found')
else:
    print('api key successfully imported')


api key successfully imported


In [57]:
def ask_perplexity(messages, model="sonar-pro", max_tokens=800, temperature=0.7):
    """
    Send a prompt to Perplexity API and return the assistant's reply.
    - prompt: User message (string)
    - model: Perplexity model to use (default: 'sonar-medium-online'); change if needed
    - max_tokens: Max tokens in response
    - temperature: Sampling temperature [0.0-2.0]
    """
    if not api_key:
        raise ValueError("PERPLEXITY_API_KEY environment variable not set.")
    
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "messages":messages,
        "max_tokens": max_tokens,
        "temperature": temperature
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]

In [37]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self, url):
        self.url =url;
        response = requests.get(url, headers);
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'No Title Found'
        for irrelevant in soup.body(['scripts','style','img','input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [39]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [40]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [41]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [55]:
# And now: call the perplexioty API.

def summarize(url):
    website = Website(url)
    response = ask_perplexity(
        messages = messages_for(website)
    )
    return response

In [47]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [59]:
display_summary("https://cnn.com")

CNN’s website provides **breaking news, latest updates, and video coverage** across a wide range of topics including US and world events, politics, business, health, entertainment, sports, science, climate, and major ongoing conflicts like the Ukraine-Russia and Israel-Hamas wars.

Key highlights and recent news include:
- Coverage of a **Minneapolis school shooting** resulting in the deaths of two children and injuries to several others.
- Updates on **Trump’s meetings regarding Gaza** and the UN’s calls for accountability over a Gaza hospital attack.
- Reports on **UN nuclear inspectors returning to Iran**, protests in Tel Aviv, and Denmark summoning the US envoy over covert influence concerns.
- Features on **SpaceX’s rocket launch**, **Naomi Osaka’s win at the US Open**, and a **bizarre engagement announcement for Taylor Swift and Travis Kelce**.
- Climate and science stories, such as the impact of climate change on ocean pH and rising cases of flesh-eating bacteria, as well as advances in robotics and space exploration.
- Political analysis on topics like **Trump’s influence on the Federal Reserve**, upcoming elections, and investigations into figures like Jeffrey Epstein.
- Lifestyle, travel, and wellness articles including sustainability tips, Mediterranean diet research, and unusual global stories.

The site also offers interactive content such as games and podcasts, and seeks user feedback to improve ad and video experiences.